# GPT From Scratch

Este repositório irá implementar o GPT 2 do zero, com o código bruto do modelo presente na pasta `src`.

O objetivo será realizar o seguinte fluxo de atividades:
* Leitura do dataset;
* Treina modelo;
* Plotagem de métricas durante treino;
* Processa modelo para gerar textos;
* Avaliação dos textos;
* Plotagem da avaliação;
* Conclusão.

---

# Preparando ambiente

Importando pacotes python para utilização do código

In [18]:
# Importando pacotes python

from os import mkdir, environ
from os.path import exists

from json import load

Importando pacotes para execução do experimento

In [2]:
from src.dataload.tokenizer import Tokenizer
from src.gptmodel.model import GPTModel

---

# Configurações para rodar o sistema

Aqui ocorrerá as constantes para o processamento do sistema:

In [3]:
RESULT_DIR = "results"

if not exists(RESULT_DIR): mkdir(RESULT_DIR)

Diretório de banco de dados:

In [4]:
DATASET_DIR = "dataset/"

Caminho final dos tokens:

In [5]:
TOKEN_TEXTS = f"{RESULT_DIR}/tokens.txt"

Caminho do melhor modelo:

In [6]:
BEST_MODEL_PATH = f"{RESULT_DIR}/best_model.pt"

Configuração do GPT:

In [7]:
GPT_CONFIGURATION = f"src/gpt_configurations.json"

Estatísticas:

In [8]:
TRAIN_RATIO = 0.8
BATCH_SIZE = 5

Configuração:

In [9]:
with open(GPT_CONFIGURATION, 'r') as f:
    CONFIGURATION = load(f)

In [10]:
CONFIGURATION_KEY = "GPT_CONFIG_124M"

In [11]:
configuration = CONFIGURATION[CONFIGURATION_KEY]

---

# Leitura do Dataset

Tokenizando todo o conjunto de dados para treinamento e salvando em um arquivo de texto:

In [12]:
tokenizer = Tokenizer(["utf-8", "latin1"])

In [13]:
tokenizer.generate_data(DATASET_DIR, TOKEN_TEXTS)

100%|██████████| 10/10 [00:00<00:00, 665.83it/s]


Carregando base de dados:

In [14]:
with open(TOKEN_TEXTS, 'r') as f:
    contents = f.read()

In [15]:
train_dataloader, test_dataloader = tokenizer.create_dataloaders(
    contents, 
    TRAIN_RATIO, 
    BATCH_SIZE, 
    configuration["vocab_size"], 
    configuration["vocab_size"]
    )

---

# Treino dos modelos

Esta seção irá ocorrer a instanciação e geração dos modelos:

In [16]:
model = GPTModel(
    vocab_size=configuration["vocab_size"],
    emb_dim=configuration["emb_dim"],
    context_length=configuration["context_length"],
    drop_rate=configuration["drop_rate"],
    n_heads=configuration["n_heads"],
    n_layers=configuration["n_layers"],
    qkv_bias=configuration["qkv_bias"]
    )

In [17]:
trained_models, final_metrics = GPTModel._train(
    model=model, 
    train_loader=train_dataloader, 
    val_loader=test_dataloader, 
    device_str="cuda",
    path=BEST_MODEL_PATH)

Training model on device: cuda


  0%|          | 0/1 [00:00<?, ?it/s]/pytorch/aten/src/ATen/native/cuda/IndexKernelUtils.cu:16: vectorized_gather_kernel: block: [1040,0,0], thread: [128,0,0] Assertion `ind >=0 && ind < ind_dim_size && "vectorized gather kernel index out of bounds"` failed.
/pytorch/aten/src/ATen/native/cuda/IndexKernelUtils.cu:16: vectorized_gather_kernel: block: [1040,0,0], thread: [129,0,0] Assertion `ind >=0 && ind < ind_dim_size && "vectorized gather kernel index out of bounds"` failed.
/pytorch/aten/src/ATen/native/cuda/IndexKernelUtils.cu:16: vectorized_gather_kernel: block: [1040,0,0], thread: [130,0,0] Assertion `ind >=0 && ind < ind_dim_size && "vectorized gather kernel index out of bounds"` failed.
/pytorch/aten/src/ATen/native/cuda/IndexKernelUtils.cu:16: vectorized_gather_kernel: block: [1040,0,0], thread: [131,0,0] Assertion `ind >=0 && ind < ind_dim_size && "vectorized gather kernel index out of bounds"` failed.
/pytorch/aten/src/ATen/native/cuda/IndexKernelUtils.cu:16: vectorized_gathe

AcceleratorError: CUDA error: device-side assert triggered
CUDA kernel errors might be asynchronously reported at some other API call, so the stacktrace below might be incorrect.
For debugging consider passing CUDA_LAUNCH_BLOCKING=1
Compile with `TORCH_USE_CUDA_DSA` to enable device-side assertions.
